In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import os
from scipy import optimize, stats, interpolate
from scipy.integrate import quad
import argparse
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
import sys
import pandas as pd
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
import healpy as hp
from copy import copy, deepcopy
# sys.path.append('BatML/')
import logging, traceback
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [2]:
NITRATES_path = '/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES/NITRATES_GIT/NITRATES'

#%cd {NITRATES_path}

In [3]:
# if the response directories are all put in the same directory
# and given as an env variable (like in this cell), then config.py will 
# be able to find them correctly as long as they keep the original names 
# (Ex: ray trace directotry is still called 'ray_traces_detapp_npy')
# These paths can also just be hardcoded into config.py
# most of these paths can also be given directly as an argument to 
# some analysis objects, like Source_Model_InOutFoV()
os.environ['NITRATES_RESP_DIR'] = '/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/'

In [4]:
from config import rt_dir
from ray_trace_funcs import RayTraces
from event2dpi_funcs import det2dpis, mask_detxy
from flux_models import Cutoff_Plaw_Flux, Plaw_Flux, get_eflux_from_model
from models import Source_Model_InOutFoV, Bkg_Model_wFlatA,\
                CompoundModel, Point_Source_Model_Binned_Rates, im_dist
from ray_trace_funcs import RayTraces
from sqlite_funcs import get_conn
from dbread_funcs import get_info_tab, get_twinds_tab
from hp_funcs import ang_sep
from coord_conv_funcs import theta_phi2imxy, imxy2theta_phi, convert_imxy2radec,\
                            convert_radec2thetaphi, convert_radec2imxy
from do_llh_inFoV4realtime2 import parse_bkg_csv
from LLH import LLH_webins
from minimizers import NLLH_ScipyMinimize_Wjacob

In [5]:
#%matplotlib inline

In [6]:
ebins0 = np.array([15.0, 24.0, 35.0, 48.0, 64.0])
ebins0 = np.append(ebins0, np.logspace(np.log10(84.0), np.log10(500.0), 5+1))[:-1]
ebins0 = np.round(ebins0, decimals=1)[:-1]
ebins1 = np.append(ebins0[1:], [350.0])
nebins = len(ebins0)
print("Nebins: ", nebins)
print("ebins0: ", ebins0)
print("ebins1: ", ebins1)

# work_dir = '/storage/work/jjd330/local/bat_data/realtime_workdir/F646018360/'
work_dir = os.path.join(NITRATES_path, 'F646018360')
conn = get_conn(os.path.join(work_dir,'results.db'))
info_tab = get_info_tab(conn)
trigger_time = info_tab['trigtimeMET'][0]
print('trigger_time: ', trigger_time)

evfname = os.path.join(work_dir,'filter_evdata.fits')
ev_data = fits.open(evfname)[1].data
GTI_PNT = Table.read(evfname, hdu='GTI_POINTING')
GTI_SLEW = Table.read(evfname, hdu='GTI_SLEW')
dmask = fits.open(os.path.join(work_dir,'detmask.fits'))[0].data
attfile = fits.open(os.path.join(work_dir,'attitude.fits'))[1].data

att_ind = np.argmin(np.abs(attfile['TIME'] - trigger_time))
att_quat = attfile['QPARAM'][att_ind]
print('attitude quaternion: ', att_quat)

ndets = np.sum(dmask==0)
print("Ndets: ", np.sum(dmask==0))

t_end = trigger_time + 1e3
t_start = trigger_time - 1e3
mask_vals = mask_detxy(dmask, ev_data)
bl_dmask = (dmask==0.)

bl_ev = (ev_data['EVENT_FLAGS']<1)&\
        (ev_data['ENERGY']<=500.)&(ev_data['ENERGY']>=14.)&\
        (mask_vals==0.)&(ev_data['TIME']<=t_end)&\
        (ev_data['TIME']>=t_start)

print("Nevents: ",np.sum(bl_ev))
ev_data0 = ev_data[bl_ev]

Nebins:  9
ebins0:  [ 15.   24.   35.   48.   64.   84.  120.  171.5 245. ]
ebins1:  [ 24.   35.   48.   64.   84.  120.  171.5 245.  350. ]
trigger_time:  646018383.1787
attitude quaternion:  [-0.03597053  0.2345147  -0.64420835  0.72712074]
Ndets:  14932
Nevents:  1367885


In [7]:
ra, dec = 233.117, -26.213 
theta, phi = convert_radec2thetaphi(ra, dec, att_quat)
print(theta, phi)
imx, imy = convert_radec2imxy(ra, dec, att_quat)
print(imx, imy)

38.54132137017975 137.65241966813443
-0.5887551341212709 -0.5366203642198198


In [8]:
flux_params = {'A':1.0, 'gamma':0.5, 'Epeak':1e2}
flux_mod = Cutoff_Plaw_Flux(E0=100.0)

In [9]:
# use rt_dir from config.py or
# rt_dir = '/path/to/ray_traces_detapp_npy/'
rt_obj = RayTraces(rt_dir)

INFO:root:0.02
INFO:root:0.002


In [10]:
rt = rt_obj.get_intp_rt(imx, imy)
print(np.shape(rt))

DEBUG:root:opening file: fwd_ray_trace_-0.60000_-0.54000_-0.58000_-0.52000_.npy
DEBUG:root:nbytes_total=95789408.0
(173, 286)


In [11]:
#%%time
# will by default use the resp directories from config.py
sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
                                rt_obj, use_deriv=True)
# or the paths can be given
# resp_tab_dname = '/path/to/resp_tabs_ebins/'
# hp_flor_resp_dname = '/path/to/hp_flor_resps/'
# comp_flor_resp_dname = '/path/to/comp_flor_resps/'
# sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
#                                 rt_obj, use_deriv=True,\
#                                 resp_tab_dname=resp_tab_dname,\
#                                 hp_flor_resp_dname=hp_flor_resp_dname,\
#                                 comp_flor_resp_dname=comp_flor_resp_dname)

In [12]:
#%%time
sig_mod.set_theta_phi(theta, phi)

INFO:root:Making new response object
(0.0, 30.48, -14.117)
(0.0, 30.48, -14.117)


/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES/NITRATES_GIT/NITRATES/response.py:1499: RuntimeWarning: invalid value encountered in true_divide
  self.E_A0s = (self.orig_photonEs[self.Einds1] - self.photonEs) /\


(36.0, 45.0)
2.652419668134428
42.34758033186557
max rt: 0.8604
DEBUG:root:Openning file: resp_by_sand_hpind_46.fits
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1428)
           2	LOAD_FAST(arg=3, lineno=1428)
           4	LOAD_ATTR(arg=0, lineno=1428)
           6	STORE_FAST(arg=5, lineno=1428)
           8	LOAD_FAST(arg=0, lineno=1429)
          10	LOAD_ATTR(arg=1, lineno=1429)
          12	LOAD_CONST(arg=1, lineno=1429)
          14	BINARY_SUBSCR(arg=None, lineno=1429)
          16	STORE_FAST(arg=6, lineno=1429)
          18	LOAD_FAST(arg=6, lineno=1430)
          20	LOAD_CONST(arg=2, lineno=1430)
          22	LOAD_CONST(arg=2, lineno=1430)
          24	LOAD_FAST(arg=5, lineno=1430)
          26	BUILD_TUPLE(arg=4, lineno=1430)
          28	STORE_FAST(arg=7, lineno=1430)
          30	LOAD_GLOBAL(arg=2, lineno=1431)
          32	LOAD_METHOD(arg=3, lineno=1431)
          34	LOAD_FAST(arg=7, lineno=1431)
          36	CALL_METHOD(arg=1, lineno=1431)
       

DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=60 nstack_initial=2)])
DEBUG:numba.core.byteflow:stack: ['$phi60.0', '$phi60.1']
DEBUG:numba.core.byteflow:dispatch pc=60, inst=FOR_ITER(arg=62, lineno=1434)
DEBUG:numba.core.byteflow:stack ['$phi60.0', '$phi60.1']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=124, stack=('$phi60.0',), blockstack=(), npush=0), Edge(pc=62, stack=('$phi60.0', '$phi60.1', '$60for_iter.3'), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=124 nstack_initial=1), State(pc_initial=62 nstack_initial=3)])
DEBUG:numba.core.byteflow:stack: ['$phi124.0']
DEBUG:numba.core.byteflow:dispatch pc=124, inst=JUMP_ABSOLUTE(arg=48, lineno=1436)
DEBUG:numba.core.byteflow:stack ['$phi124.0']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=48, stack=('$phi124.0',), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=62 nstack_initial=3), State(pc_initial=48 nstack_initial=1)])
DEBUG:numba.cor

DEBUG:numba.core.byteflow:dispatch pc=116, inst=ROT_THREE(arg=None, lineno=1436)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$phi74.1', '$phi74.2', '$new_sand_imgs76.4', '$84build_tuple.8', '$114inplace_add.24']
DEBUG:numba.core.byteflow:dispatch pc=118, inst=STORE_SUBSCR(arg=None, lineno=1436)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$phi74.1', '$phi74.2', '$114inplace_add.24', '$new_sand_imgs76.4', '$84build_tuple.8']
DEBUG:numba.core.byteflow:dispatch pc=120, inst=JUMP_ABSOLUTE(arg=72, lineno=1436)
DEBUG:numba.core.byteflow:stack ['$phi74.0', '$phi74.1', '$phi74.2']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=72, stack=('$phi74.0', '$phi74.1', '$phi74.2'), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=60 nstack_initial=2), State(pc_initial=72 nstack_initial=3)])
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=72 nstack_initial=3)])
DEBUG:numba.core.byteflow:-------------------------Prune PHIs-------------------------
DE

...

DEBUG:numba.core.interpreter:label 0:
    e0_2 = arg(0, name=e0_2)                 ['e0_2']
    pha_bins0_2 = arg(1, name=pha_bins0_2)   ['pha_bins0_2']
    pha_bins1_2 = arg(2, name=pha_bins1_2)   ['pha_bins1_2']
    $6compare_op.2 = pha_bins0_2 < e0_2      ['$6compare_op.2', 'e0_2', 'pha_bins0_2']
    $12compare_op.5 = pha_bins1_2 > e0_2     ['$12compare_op.5', 'e0_2', 'pha_bins1_2']
    $14binary_and.6 = $6compare_op.2 & $12compare_op.5 ['$12compare_op.5', '$14binary_and.6', '$6compare_op.2']
    $16return_value.7 = cast(value=$14binary_and.6) ['$14binary_and.6', '$16return_value.7']
    return $16return_value.7                 ['$16return_value.7']

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x7feb58852e20>
DEBUG:numba.core.ssa:on stmt: e0_2 = arg(0, name=e0_2)
DEBUG:numba.core.ssa:on stmt: pha_bins0_2 = arg(1, name=pha_bins0_2)
DEBUG:numba.core.ssa:on stmt: pha_bins1_2 = arg(2, name=pha_bins1_2)


DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>,
            {'$phi16.0': {('$14get_iter.5',
                           State(pc_initial=0 nstack_initial=0))},
             '$phi18.0': {('$14get_iter.5',
                           State(pc_initial=0 nstack_initial=0))},
             '$phi18.1': {('$16for_iter.2',
                           State(pc_initial=16 nstack_initial=1))}})
DEBUG:numba.core.byteflow:keep phismap: {'$phi16.0': {('$14get_iter.5', State(pc_initial=0 nstack_initial=0))},
 '$phi18.1': {('$16for_iter.2', State(pc_initial=16 nstack_initial=1))}}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>,
            {State(pc_initial=0 nstack_initial=0): {'$phi16.0': '$14get_iter.5'},
             State(pc_initial=16 nstack_initial=1): {'$phi18.1': '$16for_iter.2'}})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(in

DEBUG:numba.core.ssa:SSA violators {'c'}
DEBUG:numba.core.ssa:Fix SSA violator on var c
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._FreshVarHandler object at 0x7feb698064c0>
DEBUG:numba.core.ssa:on stmt: arr = arg(0, name=arr)
DEBUG:numba.core.ssa:on stmt: c = freevar(zero: 0)
DEBUG:numba.core.ssa:first assign: c
DEBUG:numba.core.ssa:replaced with: c = freevar(zero: 0)
DEBUG:numba.core.ssa:on stmt: $6load_global.1 = global(np: <module 'numpy' from '/Users/tparsota/opt/anaconda3/lib/python3.8/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $8load_method.2 = getattr(value=$6load_global.1, attr=nditer)
DEBUG:numba.core.ssa:on stmt: $12call_method.4 = call $8load_method.2(arr, func=$8load_method.2, args=[Var(arr, arraymath.py:167)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $14get_iter.5 = getiter(value=$12call_method.4)
DEBUG:numba.core.ssa:on stmt: $phi16.0 = $14get_iter.5
DEBUG:numba.core.ssa:on stmt: jump

DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$pha_bins0_32.0'}), (4, {'res': '$e1_24.1'}), (6, {'lhs': '$pha_bins0_32.0', 'rhs': '$e1_24.1', 'res': '$6compare_op.2'}), (8, {'res': '$pha_bins1_38.3'}), (10, {'res': '$e1_210.4'}), (12, {'lhs': '$pha_bins1_38.3', 'rhs': '$e1_210.4', 'res': '$12compare_op.5'}), (14, {'lhs': '$6compare_op.2', 'rhs': '$12compare_op.5', 'res': '$14binary_and.6'}), (16, {'retval': '$14binary_and.6', 'castval': '$16return_value.7'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.interpreter:label 0:
    e1_2 = arg(0, name=e1_2)                 ['e1_2']
    pha_bins0_3 =

initing ResponseDPI, with fname
/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/resp_tabs_ebins/drm_theta_36.0_phi_45.0_.fits
initing ResponseDPI, with fname
/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/resp_tabs_ebins/drm_theta_45.0_phi_30.0_.fits
initing ResponseDPI, with fname
/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/resp_tabs_ebins/drm_theta_45.0_phi_45.0_.fits


In [13]:
from config import solid_angle_dpi_fname
# or give the path directly
# solid_angle_dpi_fname = '/path/to/solid_angle_dpi.npy'
bkg_fname = os.path.join(work_dir,'bkg_estimation.csv')
solid_ang_dpi = np.load(solid_angle_dpi_fname)
bkg_df, bkg_name, PSnames, bkg_mod, ps_mods = parse_bkg_csv(bkg_fname, solid_ang_dpi,\
                    ebins0, ebins1, bl_dmask, rt_dir)
bkg_mod.has_deriv = False
bkg_mod_list = [bkg_mod]
Nsrcs = len(ps_mods)
if Nsrcs > 0:
    bkg_mod_list += ps_mods
    for ps_mod in ps_mods:
        ps_mod.has_deriv = False
    bkg_mod = CompoundModel(bkg_mod_list)
tmid = trigger_time
bkg_row = bkg_df.iloc[np.argmin(np.abs(tmid - bkg_df['time']))]
bkg_params = {pname:bkg_row[pname] for pname in\
            bkg_mod.param_names}
bkg_name = bkg_mod.name
bkg_params

['4U 1700-377', 'GRO J1655-40', 'GX 339-4', 'Sco X-1']
INFO:root:0.02
INFO:root:0.002
DEBUG:root:opening file: fwd_ray_trace_-0.10000_-0.48000_-0.08000_-0.46000_.npy
DEBUG:root:nbytes_total=95789408.0
DEBUG:root:opening file: fwd_ray_trace_-0.12000_-0.54000_-0.10000_-0.52000_.npy
DEBUG:root:nbytes_total=191578816.0
DEBUG:root:opening file: fwd_ray_trace_0.00000_-0.70000_0.02000_-0.68000_.npy
DEBUG:root:nbytes_total=287368224.0
DEBUG:root:opening file: fwd_ray_trace_-0.50000_-0.22000_-0.48000_-0.20000_.npy
DEBUG:root:nbytes_total=383157632.0


{'Background_bkg_rate_0': 0.0913703220701183,
 'Background_bkg_rate_1': 0.0661578002239374,
 'Background_bkg_rate_2': 0.0400898569026105,
 'Background_bkg_rate_3': 0.0394919934207801,
 'Background_bkg_rate_4': 0.0346780266835281,
 'Background_bkg_rate_5': 0.0351076584048365,
 'Background_bkg_rate_6': 0.033602492003955,
 'Background_bkg_rate_7': 0.0248610826847874,
 'Background_bkg_rate_8': 0.0176399476145076,
 'Background_flat_0': 0.0,
 'Background_flat_1': 0.0,
 'Background_flat_2': 0.1859632649428977,
 'Background_flat_3': 0.0766344244533236,
 'Background_flat_4': 0.3020855890115875,
 'Background_flat_5': 0.7611510180823338,
 'Background_flat_6': 0.8091430254293075,
 'Background_flat_7': 1.0,
 'Background_flat_8': 1.0,
 '4U 1700-377_imx': -0.0981485305770971,
 '4U 1700-377_imy': -0.4742076074486664,
 '4U 1700-377_rate_0': 0.0124901359218975,
 '4U 1700-377_rate_1': 0.0072425264886238,
 '4U 1700-377_rate_2': 0.0045295645864628,
 '4U 1700-377_rate_3': 0.0016876913514016,
 '4U 1700-377_r

In [14]:
pars_ = {}
pars_['Signal_theta'] = theta
pars_['Signal_phi'] = phi
for pname,val in list(bkg_params.items()):
    pars_[bkg_name+'_'+pname] = val
for pname,val in list(flux_params.items()):
    pars_['Signal_'+pname] = val
pars_

{'Signal_theta': 38.54132137017975,
 'Signal_phi': 137.65241966813443,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_0': 0.0913703220701183,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_1': 0.0661578002239374,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_2': 0.0400898569026105,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_3': 0.0394919934207801,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_4': 0.0346780266835281,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_5': 0.0351076584048365,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_6': 0.033602492003955,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_7': 0.0248610826847874,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_8': 0.0176399476145076,
 'Background+4U 1700-377+GRO J1655-40+G

In [15]:
comp_mod = CompoundModel([bkg_mod, sig_mod])

sig_miner = NLLH_ScipyMinimize_Wjacob('')

sig_llh_obj = LLH_webins(ev_data0, ebins0, ebins1, bl_dmask, has_err=True)

sig_llh_obj.set_model(comp_mod)

sig_miner.set_llh(sig_llh_obj)

In [16]:
fixed_pnames = list(pars_.keys())
fixed_vals = list(pars_.values())
trans = [None for i in range(len(fixed_pnames))]
sig_miner.set_trans(fixed_pnames, trans)
sig_miner.set_fixed_params(fixed_pnames, values=fixed_vals)
sig_miner.set_fixed_params(['Signal_A'], fixed=False)

In [17]:
#%%time
flux_params['gamma'] = 0.8
flux_params['Epeak'] = 350.0
sig_mod.set_flux_params(flux_params)

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1110)
           2	LOAD_FAST(arg=0, lineno=1110)
           4	LOAD_ATTR(arg=0, lineno=1110)
           6	UNPACK_SEQUENCE(arg=3, lineno=1110)
           8	STORE_FAST(arg=2, lineno=1110)
          10	STORE_FAST(arg=3, lineno=1110)
          12	STORE_FAST(arg=4, lineno=1110)
          14	LOAD_GLOBAL(arg=1, lineno=1111)
          16	LOAD_METHOD(arg=2, lineno=1111)
          18	LOAD_FAST(arg=2, lineno=1111)
          20	LOAD_FAST(arg=4, lineno=1111)
          22	BUILD_TUPLE(arg=2, lineno=1111)
          24	CALL_METHOD(arg=1, lineno=1111)
          26	STORE_FAST(arg=5, lineno=1111)
          28	LOAD_GLOBAL(arg=3, lineno=1113)
          30	LOAD_FAST(arg=2, lineno=1113)
          32	CALL_FUNCTION(arg=1, lineno=1113)
          34	GET_ITER(arg=None, lineno=1113)
>         36	FOR_ITER(arg=70, lineno=1113)
          38	STORE_FAST(arg=6, lineno=1113)
          40	LOAD_GLOBAL(arg=3, lineno=1114)
          42	LOAD_FAST(arg=4, 

DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1', '$phi50.2']
DEBUG:numba.core.byteflow:dispatch pc=52, inst=LOAD_GLOBAL(arg=3, lineno=1115)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1']
DEBUG:numba.core.byteflow:dispatch pc=54, inst=LOAD_FAST(arg=3, lineno=1115)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1', '$52load_global.3']
DEBUG:numba.core.byteflow:dispatch pc=56, inst=CALL_FUNCTION(arg=1, lineno=1115)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1', '$52load_global.3', '$NphotonEs54.4']
DEBUG:numba.core.byteflow:dispatch pc=58, inst=GET_ITER(arg=None, lineno=1115)
DEBUG:numba.core.byteflow:stack ['$phi50.0', '$phi50.1', '$56call_function.5']
DEBUG:numba.core.byteflow:end state. edges=[Edge(pc=60, stack=('$phi50.0', '$phi50.1', '$58get_iter.6'), blockstack=(), npush=0)]
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=36 nstack_initial=1), State(pc_initial=60 nstack_initial=3)])
DEBUG:numba.core.byteflow:pending: deque([State(pc_initi

DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>,
            {'$phi104.0': {('$phi50.0', State(pc_initial=50 nstack_initial=3)),
                           ('$phi62.0', State(pc_initial=62 nstack_initial=4))},
             '$phi104.1': {('$phi50.1', State(pc_initial=50 nstack_initial=3)),
                           ('$phi62.1', State(pc_initial=62 nstack_initial=4))},
             '$phi106.0': {('$phi104.0',
                            State(pc_initial=104 nstack_initial=2)),
                           ('$phi38.0', State(pc_initial=38 nstack_initial=2))},
             '$phi36.0': {('$34get_iter.15',
                           State(pc_initial=0 nstack_initial=0)),
                          ('$phi104.0', State(pc_initial=104 nstack_initial=2)),
                          ('$phi38.0', State(pc_initial=38 nstack_initial=2))},
             '$phi38.0': {('$34get_iter.15',
                           State(pc_initial=0 nstack_initial=0)),
                          ('$phi1

DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$resp_dpi2.0'}), (4, {'item': '$resp_dpi2.0', 'res': '$4load_attr.1'}), (6, {'iterable': '$4load_attr.1', 'stores': ['$6unpack_sequence.2', '$6unpack_sequence.3', '$6unpack_sequence.4'], 'tupleobj': '$6unpack_sequence.5'}), (8, {'value': '$6unpack_sequence.2'}), (10, {'value': '$6unpack_sequence.3'}), (12, {'value': '$6unpack_sequence.4'}), (14, {'res': '$14load_global.6'}), (16, {'item': '$14load_global.6', 'res': '$16load_method.7'}), (18, {'res': '$ndets18.8'}), (20, {'res': '$Nphabins20.9'}), (22, {'items': ['$ndets18.8', '$Nphabins20.9'], 'res': '$22build_tuple.10'}), (24, {'func': '$16load_method.7', 'args': ['$22build_tuple.10'], 'res': '$24call_method.11'}), (26, {'value': '$24call_method.11'}), (28, {'res': '$28load_global.12'}), (30, {'res': '$ndets30.13'}), (32, {'func': 

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x7feb48700d60>
DEBUG:numba.core.ssa:on stmt: resp_dpi = arg(0, name=resp_dpi)
DEBUG:numba.core.ssa:on stmt: photon_fluxes = arg(1, name=photon_fluxes)
DEBUG:numba.core.ssa:on stmt: $4load_attr.1 = getattr(value=resp_dpi, attr=shape)
DEBUG:numba.core.ssa:on stmt: $6unpack_sequence.5 = exhaust_iter(value=$4load_attr.1, count=3)
DEBUG:numba.core.ssa:on stmt: $6unpack_sequence.2 = static_getitem(value=$6unpack_sequence.5, index=0, index_var=None, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $6unpack_sequence.3 = static_getitem(value=$6unpack_sequence.5, index=1, index_var=None, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: $6unpack_sequence.4 = static_getitem(value=$6unpack_sequence.5, index=2, index_var=None, fn=<built-in function getitem>)
DEBUG:numba.core.ssa:on stmt: ndets = $6unpack_sequence.2
DEBUG:numba.core.ssa:on stmt: N

DEBUG:numba.core.ssa:SSA violators set()


In [18]:
t0 = trigger_time - 0.512
t1 = t0 + 2.048
sig_llh_obj.set_time(t0, t1)

In [19]:
#%%time
pars, nllh, res = sig_miner.minimize()

In [20]:
print(res)
print(nllh)
print(pars)

[      fun: 45894.070681000456
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00239975])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 8
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([0.006875])]
[45894.070681000456]
[[0.006875003655224625]]


In [21]:
#%%time
pars_['Signal_A'] = 1e-10
bkg_nllh = -sig_llh_obj.get_logprob(pars_)
print (bkg_nllh)
sqrtTS = np.sqrt(2.*(bkg_nllh - nllh[0]))
print (sqrtTS)

46038.743327207194
17.01015262757734


In [22]:
vars(sig_mod)

{'fmodel': <flux_models.Cutoff_Plaw_Flux at 0x7feb88462c40>,
 'ebins': [array([ 15. ,  24. ,  35. ,  48. ,  64. ,  84. , 120. , 171.5, 245. ]),
  array([ 24. ,  35. ,  48. ,  64. ,  84. , 120. , 171.5, 245. , 350. ])],
 'ebins0': array([ 15. ,  24. ,  35. ,  48. ,  64. ,  84. , 120. , 171.5, 245. ]),
 'ebins1': array([ 24. ,  35. ,  48. ,  64. ,  84. , 120. , 171.5, 245. , 350. ]),
 'resp_dname': '/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/resp_tabs_ebins',
 'flor_resp_dname': '/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/hp_flor_resps',
 'comp_flor_resp_dname': '/Users/tparsota/Documents/BAT_SCRIPTS/NITRATES_BAT_RSP_FILES/comp_flor_resps',
 '_name': 'Signal',
 '_bl_dmask': array([[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True,  True, False],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True, False, False, ...,  True,  

In [23]:
normed_comp_flor_rate_dpis, normed_photoe_rate_dpis,normed_rate_dpis, normed_err_rate_dpis

NameError: name 'normed_comp_flor_rate_dpis' is not defined

In [ ]:
vars(sig_mod.resp_obj)

In [ ]:
resp_files, phis2use, wts